In [1]:
# Import necessary packages
import stim
import numpy as np
from sympy import *
import copy
import galois
from typing import List, Tuple
from itertools import chain, combinations

In [52]:
class PauliSummand:
    def __init__(self, phase: complex, pauli: stim.PauliString):
        """ 
        Representation for single element of 'PauliSum'

        Params:
        -------
        phase - Phase of the 'PauliSummand' instance
        pauli - Pauli operator of the 'PauliSummand' instance
        """
        self.phase = phase 
        self.pauli = pauli 
        
        # Store basis paulis for each 'PauliSummand' instance
        self.bases = []

    def get_phase(self):
        """ 
        Get phase of PauliSummand

        Returns:
        --------
        self.phase
        """
        return self.phase 

    def set_phase(self, phase: complex):
        """ 
        Set phase of PauliSummand 

        Params:
        -------
        phase - Phase to set to of 'PauliSummand' instance
        """
        self.phase = phase

    def get_pauli(self):
        """ 
        Get pauli of PauliSummand

        Returns:
        --------
        self.pauli
        """
        return self.pauli

    def set_pauli(self, pauli: stim.PauliString):
        """ 
        Set pauli of PauliSummand 

        Params:
        -------
        pauli - Pauli to set to of 'PauliSummand' instance
        """
        self.pauli = pauli

    def get_bases(self):
        """ 
        Get list of basis elements that generate 'PauliSummand' instance

        Returns:
        --------
        self.bases
        """
        return self.bases 

    def add_to_basis(self, basis_ind: int):
        """ 
        Add basis index corresponding to index of basis element in overall list of 
        Pauli bases

        Params:
        -------
        basis_ind - Index of basis element in overall list of Pauli bases
        """
        self.bases.append(basis_ind)
        
    def get_pauli_str(self, pauli:stim.PauliString):
        """ 
        Extract just pauli string from stim Pauli representation
        """
        pauli_str = ""
        pauli_plus_phase = str(pauli)
        pauli_plus_phase_len = len(pauli_plus_phase)
        count = pauli_plus_phase_len - 1
        while (pauli_plus_phase[count] == '_' or pauli_plus_phase[count] == 'X' or 
            pauli_plus_phase[count] == 'Y' or pauli_plus_phase[count] == 'Z'):
            if (pauli_plus_phase[count] == '_'):
                pauli_str = "I" + pauli_str
            else:
                pauli_str = pauli_plus_phase[count] + pauli_str 
            count -= 1
        return pauli_str

    def __str__(self):
        pauli_str = ""
        pauli_plus_phase = str(self.pauli)
        pauli_plus_phase_len = len(pauli_plus_phase)
        count = pauli_plus_phase_len - 1
        while (pauli_plus_phase[count] == '_' or pauli_plus_phase[count] == 'X' or 
            pauli_plus_phase[count] == 'Y' or pauli_plus_phase[count] == 'Z'):
            if (pauli_plus_phase[count] == '_'):
                pauli_str = "I" + pauli_str
            else:
                pauli_str = pauli_plus_phase[count] + pauli_str 
            count -= 1
        return "[" + str(self.phase) + " " + pauli_str + "]"
    
    def get_matrix_rep(self):
        result = np.eye(1)
        p_I = np.array([
            [1,0],
            [0,1]
        ])
        p_X = np.array([
            [0,1],
            [1,0]
        ])
        p_Y = np.array([
            [0, -1j],
            [1j, 0]
        ])
        p_Z = np.array([
            [1,0],
            [0,-1]
        ])
        pauli_dict = {'I':p_I, 'X':p_X, 'Y':p_Y, 'Z': p_Z}
        for p_op in self.get_pauli_str(self.pauli):
            result = np.kron(result, pauli_dict[p_op])
        result = self.get_phase() * result
        return result

class PauliSum: 
    def __init__(self, *argv):
        """ 
        Representation of contraction of QCSAT instance
        """
        if len(argv) == 0:
            self.sum = []
            self.bases = []
            self.num_bases = 0
            self.summand_to_bases = {}
        else:
            self.sum = [argv[0]]
            self.bases = [argv[0]]
            self.num_bases = 1
            self.summand_to_bases = {str(argv[0].get_pauli()):[0]}

    def get_sum(self):
        """ 
        Get PauliSum in list representation
        """
        return self.sum 

    def set_sum(self, sum):
        self.sum = sum

    def get_bases(self):
        """ 
        Get bases for PauliSum
        """
        return self.bases 

    def get_basis_elem(self, pos: int):
        """
        Get basis element
        """ 
        try:
            return self.bases[pos]
        except IndexError as e:
            print(f"{e} " + str(self.bases) + " - Bases; " + str(self) + " - Sum; " + str(pos) + " - Position")


    def set_basis_elem(self, pos: int, basis_elem: stim.PauliString):
        """ 
        Set a particular basis element
        """
        self.bases[pos] = basis_elem

    def set_bases(self, basis_list: List):
        """ 
        Set basis for PauliSum

        Params:
        -------
        basis_list - List of bases for PauliSum
        """
        self.bases = basis_list 
        self.set_num_bases(len(self.bases))

    def get_num_bases(self):
        """ 
        Get number of bases in PauliSum
        """
        return self.num_bases

    def set_num_bases(self, basis_count: int):
        """ 
        Set number of basis elements in PauliSum
        """
        self.num_bases = basis_count

    def add_to_bases(self, basis_elem: stim.PauliString):
        """
        Add a basis element to self.bases
        """ 
        self.bases.append(basis_elem)
        self.incr_num_bases()

    def incr_num_bases(self):
        """ 
        Increment 'num_bases' variable
        """
        self.num_bases += 1

    def __str__(self):
        str_out = ""
        for summand in self.sum:
            str_out += str(summand)
            str_out += " + "
        return str_out[:len(str_out) - 3]

    def add_sum_to_sum(self, p):
        """ 
        Add a PauliSum instance to 'self'

        Params:
        -------
        p - PauliSum instance to be added
        """
        self.sum += p.sum 

    def add_summand(self, s: PauliSummand):
        """ 
        Add a PauliSummand to 'self'

        Params:
        -------
        s - PauliSummand instance to be added 
        """
        self.sum.append(s)

    def get_summand_bases_dict(self):
        """ 
        Get 'summand_to_bases' dict
        """
        return self.summand_to_bases

    def set_dict(self, dict):
        """ 
        Set 'summand_to_bases" dict
        """
        self.summand_to_bases = dict

    def retrieve_gen_basis(self, dict_key: str):
        """ 
        Returns generating set of Paulis for given PauliSummand
        """
        return self.summand_to_bases[dict_key] 

    def append_gen_basis(self, dict_key: str, gen_basis: int):
        """ 
        Appends a generating basis element to list associated with particular PauliSummand
        """
        if (gen_basis not in self.summand_to_bases[dict_key]):
            self.summand_to_bases[dict_key].append(gen_basis)

    def find_like_terms_sum(self):
        """ 
        Find terms with same Pauli String in PauliSum
        """
        p_sum = self.get_sum()
        like_terms_dict = {str(p_sum[i].get_pauli()):[] for i in range(len(p_sum))}
        p_str = [str(p_sum[i].get_pauli()) for i in range(len(p_sum))]
        for i in range(len(p_sum)):
            like_terms_dict[p_str[i]].append(i)
        return like_terms_dict

    def get_pauli_str(self, pauli:stim.PauliString):
        """ 
        Extract just pauli string from stim Pauli representation
        """
        pauli_str = ""
        pauli_plus_phase = str(pauli)
        pauli_plus_phase_len = len(pauli_plus_phase)
        count = pauli_plus_phase_len - 1
        while (pauli_plus_phase[count] == '_' or pauli_plus_phase[count] == 'X' or 
            pauli_plus_phase[count] == 'Y' or pauli_plus_phase[count] == 'Z'):
            if (pauli_plus_phase[count] == '_'):
                pauli_str = "I" + pauli_str
            else:
                pauli_str = pauli_plus_phase[count] + pauli_str 
            count -= 1
        return pauli_str

    def combine_like_terms_sum(self):
        """ 
        Combine like terms of PauliSum
        """
        like_terms_dict = self.find_like_terms_sum()
        #print(like_terms_dict)
        new_p_sum = []
        
        p_sum = self.get_sum()
        p_str = [str(p_sum[i].get_pauli()) for i in range(len(p_sum))]
        for pauli in like_terms_dict:
            new_pauli = stim.PauliString(pauli)
            new_summand = None
            if (len(like_terms_dict[pauli]) == 1):
                #print(p_str.index(like_terms_dict[pauli][0]))
                new_p_sum.append(p_sum[like_terms_dict[pauli][0]])
            else:
                new_phase = 0
                for ind in like_terms_dict[pauli]:
                    new_phase += p_sum[ind].get_phase()
                if (new_phase == 0):
                    continue 
                new_summand = PauliSummand(new_phase, new_pauli)
                new_p_sum.append(new_summand)
        self.set_sum(new_p_sum)

    def check_basis_dependence(self, added_basis: bool, num_qubits: int):
        """ 
        Checks the linear dependence of the current basis set

        Params:
        -------
        added_basis - Boolean that tells us whether a basis element was added when T gate was applied
        num_qubits - Number of qubits of our system
        """
        if (added_basis == False):
            return 

        is_dependence = False
        curr_basis = self.get_bases()
        if(len(curr_basis) == 1):
            return
        #print("Before: " + str(curr_basis))
        added_basis = curr_basis[-1]
        curr_basis_cp = list(curr_basis[:-1])
        basis_subsets = list(chain.from_iterable(combinations(curr_basis_cp, r) for r in range(len(curr_basis_cp) + 1)))[1:]
        for subset in basis_subsets:
            product = stim.PauliString(num_qubits)
            for basis in subset:
                product = product * basis
            if (self.get_pauli_str(product) == self.get_pauli_str(added_basis)):
                is_dependence = True
                curr_basis = curr_basis_cp
        #print("After: " + str(curr_basis))

        self.set_bases(curr_basis)
        return is_dependence

    def update_summand_bases_dict(self, gen_basis: int, basis_to_add: int):
        """ 
        Update 'summand_to_bases' in order to reflect transformation of basis
        """
        for key in self.summand_to_bases:
            if (gen_basis in self.retrieve_gen_basis(key)):
                self.append_gen_basis(key, basis_to_add)

    def add_to_summand_bases_dict(self, summand: str, basis_num: int):
        """ 
        Add a PauliSummand to 'summand_to_bases' dict 
        """
        if (summand in self.summand_to_bases and basis_num not in self.summand_to_bases[summand]):
            self.summand_to_bases[summand].append(basis_num)
        elif (summand in self.summand_to_bases and basis_num in self.summand_to_bases[summand]):
            self.summand_to_bases[summand].remove(basis_num)
        else:
            self.summand_to_bases[summand] = [basis_num]

    def set_summand_bases_dict(self, summand:str, bases: List):
        """ 
        Set generating set list for PauliSummand
        """
        self.summand_to_bases[summand] = bases

    def transform_basis(self, t_gate_loc: int):
        """ 
        Transform basis so that T-gate can be appropriately applied to basis paulis. 
        We want only two bases acting non-trivially at the position where the T-gate 
        is to be applied.

        Params:
        -------
        t_gate_loc - Qubit location where T-gate is being applied
        """
        pauli_dict = {0: '_', 1: 'X', 2: 'Y', 3: 'Z'}
        pauli_column = [pauli_dict[p.__getitem__(t_gate_loc)] for p in self.bases]
        new_basis_paulis = []
        x_count = 0
        x_pos = 0
        y_count = 0
        y_pos = 0
        """ 
        x_pos = [i for i in range(len(pauli_column)) if pauli_column[i] == 'X'][0]
        """
        for i in range(len(pauli_column)):
            if (pauli_column[i] == '_' or pauli_column[i] == 'Z'):
                new_basis_paulis.append(self.get_basis_elem(i))
            if (pauli_column[i] == 'X'):
                if (x_count == 0):
                    new_basis_paulis.append(self.get_basis_elem(i))
                    x_pos = i
                else:
                    new_basis_paulis.append(self.get_basis_elem(x_pos) * self.get_basis_elem(i))
                    #self.update_summand_bases_dict(i, x_pos)
                x_count += 1

            if (pauli_column[i] == 'Y'):
                if (y_count == 0):
                    new_basis_paulis.append(self.get_basis_elem(i))
                    y_pos = i
                else:
                    new_basis_paulis.append(self.get_basis_elem(y_pos) * self.get_basis_elem(i))
                    #self.update_summand_bases_dict(i, y_pos)
                y_count += 1

        x_pos_cp = x_pos
        y_pos_cp = y_pos

        #print("X and Y count is: " + str((x_count, y_count)))

        if (x_count >= 1 and y_count >= 1):
            new_basis_paulis[y_pos] = new_basis_paulis[x_pos] * new_basis_paulis[y_pos]
            y_pos_cp = None
            #self.update_summand_bases_dict(y_pos, x_pos)

        if (x_count == 0 and y_count >= 1):
            x_pos_cp = None

        if (x_count == 0 and y_count == 0):
            x_pos_cp = None
            y_pos_cp = None

        self.set_bases(new_basis_paulis)

        return x_pos_cp, y_pos_cp

    def apply_t_gate(self, num_qubits: int, t_gate_loc: int = 0):
        """ 
        Simulate application of T gate on PauliSum based on which qubit T gate is 
        being applied to 

        Params:
        -------
        num_qubits - Number of qubits
        t_gate_loc - Location where T gate is being applied
        """
        phase_term = 1/np.sqrt(2)
        pauli_dict = {0: '_', 1: 'X', 2: 'Y', 3: 'Z'}
        added = False
        added_basis = False
        is_empty = True
        # new_sum = PauliSum()
        X_term = stim.PauliString('_' * t_gate_loc + 'X' + '_' * (num_qubits - t_gate_loc - 1))
        Y_term = stim.PauliString('_' * t_gate_loc + 'Y' + '_' * (num_qubits - t_gate_loc - 1))

        # Update Pauli Basis
        x_pos, y_pos = self.transform_basis(t_gate_loc)
        #print("X_pos and Y_pos are: " + str((x_pos, y_pos)))
        #print("T-gate location: " + str(t_gate_loc))
        pauli_column = [pauli_dict[p.__getitem__(t_gate_loc)] for p in self.bases]
        for i in range(len(pauli_column)):
            if (pauli_column[i] == '_' or pauli_column[i] == 'Z'):
                continue 
            elif (pauli_column[i] == 'X'):
                new_basis = self.get_basis_elem(i) * X_term * Y_term
                self.add_to_bases(new_basis)
                #self.add_to_summand_bases_dict(str(new_basis), self.get_num_bases()) 
            else:
                new_basis = self.get_basis_elem(i) * Y_term * X_term 
                self.add_to_bases(new_basis)
                #self.add_to_summand_bases_dict(str(new_basis), self.get_num_bases())
                self.set_basis_elem(i, self.get_basis_elem(i) * -1)

        #print("The current basis is: " + str(self.get_bases()))

        if (x_pos != None or y_pos != None):
            added_basis = True 
        is_dependence = self.check_basis_dependence(added_basis, num_qubits)
        
        # Update Pauli Summands
        for summand in self.sum:
            #print("Pauli and t_gate_loc: " + str(summand.get_pauli().__getitem__(t_gate_loc)))
            if (pauli_dict[summand.get_pauli().__getitem__(t_gate_loc)] == '_' or pauli_dict[summand.get_pauli().__getitem__(t_gate_loc)] == 'Z'):
                #print("Identity or Z")
                continue
            elif (pauli_dict[summand.get_pauli().__getitem__(t_gate_loc)] == 'X'):
                #print("X")
                new_summand = PauliSummand(phase_term * summand.get_phase(), summand.get_pauli() * X_term * Y_term)
                summand.set_phase(summand.get_phase() * phase_term)
                if (added == False):
                    new_sum = PauliSum(new_summand)
                    is_empty = False
                    added = True 
                else:
                    new_sum.add_summand(new_summand)
                
            else:
                #print("Y")
                new_summand = PauliSummand(phase_term * summand.get_phase(), summand.get_pauli() * Y_term * X_term)
                summand.set_phase(-1 * 
                summand.get_phase() * phase_term)
                if (added == False):
                    new_sum = PauliSum(new_summand)
                    is_empty = False
                    added = True 
                else:
                    new_sum.add_summand(new_summand)

            """
            if (new_summand.get_pauli() in self.get_bases()):
                self.add_to_summand_bases_dict(str(new_summand.get_pauli()), self.get_bases().index(new_summand.get_pauli()))
            else:
                gen_set = list(self.retrieve_gen_basis(str(summand.get_pauli())))
                if (x_pos in gen_set):
                    gen_set.remove(x_pos)
                    gen_set.append(self.get_bases().index(self.get_bases()[x_pos] * X_term * Y_term))
                    self.set_summand_bases_dict(str(new_summand.get_pauli()), gen_set)
                if (y_pos in gen_set):
                    gen_set.remove(y_pos)
                    gen_set.append(self.get_bases().index(-1 * self.get_bases()[y_pos] * Y_term * X_term))
                    self.set_summand_bases_dict(str(new_summand.get_pauli()), gen_set)
            """   

        if (not is_empty):
            self.add_sum_to_sum(new_sum)

        self.combine_like_terms_sum()

        if (is_dependence):
            new_basis = [self.get_sum()[0].pauli]
            self.set_bases(new_basis) 

    def get_bin_sym_form(self, p_str: str) -> Tuple:
        """
        Convert Pauli string into Binary symplectic form

        Params:
        -------
        p_str 
        """ 
        X_op = []
        Z_op = []
        pos = len(p_str) - 1
        while(p_str[pos] == '_' or p_str[pos] == 'I' or p_str[pos] == 'X' or p_str[pos] == 'Y' or p_str[pos] == 'Z'):
            if (p_str[pos] == '_' or p_str[pos] == 'I'):
                X_op = [0] + X_op 
                Z_op = [0] + Z_op 
            elif (p_str[pos] == 'X'):
                X_op = [1] + X_op 
                Z_op = [0] + Z_op 
            elif (p_str[pos] == 'Z'):
                X_op = [0] + X_op 
                Z_op = [1] + Z_op 
            else:
                X_op = [1] + X_op 
                Z_op = [1] + Z_op 
            pos -= 1

        X_op = np.array(X_op)
        Z_op = np.array(Z_op)
        op = np.array(np.hstack((X_op, Z_op)))
        return op 

    def basis_matrix(self) -> np.array:
        bin_sym_list = []
        for basis in self.get_bases():
            bin_sym_list.append(self.get_bin_sym_form(str(basis)))

        return np.vstack(tuple(bin_sym_list))

    def find_summand_bases_mapping(self) -> dict: 
        """ 
        Find the mapping between summands of our PauliSum and our bases
        """
        GF = galois.GF(2)
        mapping_dict = {}
        p_sum_list = self.get_sum()
        mat = self.basis_matrix().T
        aug_mat = Matrix(np.hstack((mat, np.eye(len(mat), dtype=int))))
        aug_mat_red = GF(np.mod(np.array(aug_mat.rref()[0], dtype=int),2))[:,len(mat[0]):]
        for summand in p_sum_list:
            bin_summand = self.get_bin_sym_form(str(summand.get_pauli())).reshape(1,-1).T
            mapping = (GF(aug_mat_red) @ GF(bin_summand))[:len(mat[0])]
            mapping_dict[str(summand.get_pauli())] = mapping 
        return mapping_dict

    def generate_succinct_basis_term(self, k: int, num_bases: int) -> stim.PauliString:
        """ 
        Generate new basis element A^(k)

        Params:
        -------
        k - Index related to the k-th position in Pauli String
        num_bases - Number of basis elements
        """
        gamma = lambda i,j: self.get_basis_elem(i).commutes(self.get_basis_elem(j))
        new_basis_elem = stim.PauliString(num_bases)
        for i in range(k):
            if (gamma(i,k) == False):
                #pauli_str = '_' * i + 'X' + '_' * (k - i - 1) + 'Z' + '_' * (num_qubits - k - 1)
                pauli_str = '_' * i + 'X' + '_' * (k - i - 1) + 'Z'
            else:
                #pauli_str = '_' * k + 'Z' + '_' * (num_qubits - k - 1)
                pauli_str = '_' * k + 'Z'
            new_basis_elem *= stim.PauliString(pauli_str)
        #new_basis_elem = stim.PauliString(pauli_str)

        return new_basis_elem

    def generate_succinct_basis_term_V2(self, k: int, num_bases: int) -> stim.PauliString:
        gamma = lambda i,j: self.get_basis_elem(i).commutes(self.get_basis_elem(j))
        new_basis_elem = stim.PauliString('_' * k + 'Z' + (num_bases - k - 1) * '_')
        for i in range(k):
            if (gamma(i,k) == False):
                pauli_str = '_' * i + 'X' + '_' * (k - i)
            else:
                pauli_str = '_' * k
            new_basis_elem *= stim.PauliString(pauli_str)
        return new_basis_elem


    def succinct_basis(self, num_qubits: int):
        """
        Derive a more succinct basis as prescribed by description of Lemma 17

        Params:
        -------
        num_qubits - Number of qubits
        """ 
        b = self.num_bases
        #print(self.bases)
        new_basis = [stim.PauliString("Z" + "_" * (b - 1))]
        for i in range(1, b):
            new_basis.append(self.generate_succinct_basis_term_V2(i, b))
        #self.set_bases(new_basis)
        return new_basis

    def get_succinct_summand(self, summand: PauliSummand, mapping_dict: dict, num_qubits: int, succinct_basis):
        """ 
        Find succinct representation of summand in terms of succinct basis
        """
        summand_str = str(summand.get_pauli())
        basis_elems = mapping_dict[summand_str]
        new_phase = summand.get_phase()
        new_pauli = stim.PauliString(len(succinct_basis))
        old_pauli = stim.PauliString(num_qubits)
        old_bases = self.get_bases()
        for i in range(len(basis_elems)):
            if (basis_elems[i] == 1):
                new_pauli *= succinct_basis[i]
                old_pauli *= old_bases[i] 
        print(old_pauli.sign / new_pauli.sign)
        phase_factor = old_pauli.sign/new_pauli.sign
        new_summand = PauliSummand(new_phase * phase_factor, new_pauli)
        return new_summand

    def succinct_p_sum(self, num_qubits: int):
        """ 
        Return p_sum in terms of succinct basis
        """
        self.summand_to_bases = self.find_summand_bases_mapping()
        print(self.summand_to_bases)
        succinct_basis = self.succinct_basis(num_qubits)
        curr_sum = self.get_sum()
        new_sum = PauliSum()
        for summand in curr_sum:
            new_summand = self.get_succinct_summand(summand, self.summand_to_bases, num_qubits, succinct_basis)
            new_sum.add_summand(new_summand)
        return new_sum, succinct_basis

    def compute_eig(self, num_terms: int):
        result = np.zeros((2**num_terms, 2**num_terms), dtype=complex)
        for summand in self.sum:
            #result += summand.get_matrix_rep() * summand.get_pauli().sign
            result += summand.get_matrix_rep()
        return np.linalg.eig(result)

    def compute_min_eig_eigvec(self, num_qubits: int):
        eigs = self.compute_eig(num_qubits)
        e_vals = eigs[0]
        e_vecs = eigs[1]
        min_pos = e_vals.argmin()
        return (e_vals[min_pos], e_vecs[:,min_pos])
        
    
    def apply_tableau(self, t: stim.Tableau):
        """ 
        Apply current stabilizer tableau on PauliSum instance

        Params:
        -------
        t - stabilizer tableau
        """
        # Apply tableau to PauliSum
        for summand in self.sum:
            new_summand = t(summand.get_pauli())
            summand.set_phase(summand.get_phase() * new_summand.sign)
            summand.set_pauli(new_summand)

        self.combine_like_terms_sum()

        # Apply tableau to PauliSum basis
        self.set_bases([t(p) for p in self.get_bases()])

        gen_basis_list = []

        # Update keys of dictionary
        new_summand_to_bases = {}
        for key in list(self.summand_to_bases):
            new_key = str(t(stim.PauliString(key)))
            new_summand_to_bases[new_key] = self.summand_to_bases.get(key)
        self.set_dict(new_summand_to_bases)
            

In [26]:
def sim_circ(num_qubits: int, circ_depth: int, num_t_gate: int) -> PauliSum:
    """
    Calculates U^{\dag}ZU for arbitrary Clifford + T circuit with a 
    single T gate acting WLOG on the first qubit

    Parameters:
    -----------
    * num_tot_qubits - Number of total qubits in system
    * circ_depth - Number of layers of operators in our circuit
    * num_t_gate - Number of t-gates to be inserted in our circuit

    Returns:
    ---------
    pauli_sum - Instance of PauliSum class
    """

    t = stim.Tableau(num_qubits)
    gates_applied = []
    basis_paulis = []
    t_gate_layer_loc = [] # List of circuit layer positions corresponding to T-gate application
    t_gate_qubit_loc = [] # Which qubit to apply the t-gate to.
    t_gate_count = 0;
    curr_layer = 0
    write_str = ""

    pauli_z_string = "Z" + "_" * (num_qubits - 1)
    z = stim.PauliString(pauli_z_string)
    z_phase = 1 + 0j
    z_summand = PauliSummand(z_phase, z)
    pauli_sum = PauliSum(z_summand)
    basis_paulis.append(z)
    pauli_sum.set_bases(basis_paulis)

    # Create list of circuit depth positions where we would like to apply T-gate
    for _ in range(num_t_gate):
        num_to_add = np.random.randint(0, circ_depth)
        if (num_to_add in t_gate_layer_loc):
            continue
        else:
            t_gate_layer_loc.append(num_to_add)
    
    # Create list of qubit positions where we would like to apply T-gate in each layer
    for _ in range(circ_depth):
        num_to_add = np.random.randint(0, num_qubits)
        t_gate_qubit_loc.append(num_to_add)

    # Need to sort this array so that application of T-gates makes chronological sense with elements of 't_gate_layer_loc'
    t_gate_layer_loc = np.sort(t_gate_layer_loc)
    #print("Qubit location of T-gates: " + str(t_gate_qubit_loc))
    #print("Layer location of T-gates: " + str(t_gate_layer_loc))
    for i in range(circ_depth):
        write_str += "Layer " + str(i) + ":" + str(pauli_sum) + "\n"
        write_str += "Generating Bases: " + str(pauli_sum.get_summand_bases_dict()) + "\n"
        write_str += "Curr Basis: " + str(pauli_sum.get_bases()) + "\n"
        if (t_gate_count < len(t_gate_layer_loc) and i == t_gate_layer_loc[t_gate_count]):
            pauli_sum.apply_t_gate(num_qubits, t_gate_qubit_loc[curr_layer])

            #print("Intermediate T gate: " + str(pauli_sum))

            t_gate_count += 1
            t = stim.Tableau(num_qubits)
            gate_applied = 'T' + ' ' + str(t_gate_qubit_loc[curr_layer])
            gates_applied.append('T' + ' ' + str(t_gate_qubit_loc[curr_layer]))
            
        else:
            gate_to_add = stim.Tableau.random(num_qubits)
            pauli_sum.apply_tableau(gate_to_add)
            #print("Intermediate Clifford gate: " + str(pauli_sum))
            gate_applied = str(gate_to_add)
            gates_applied.append(gate_to_add)

        curr_layer += 1
        write_str += gate_applied + "\n"
        write_str += "\n"
    return pauli_sum, write_str

In [27]:
def get_commutation_matrix(bases: List):
    comm_mat = np.eye(len(bases))
    for i in range(len(bases)):
        for j in range(len(bases)):
            if (i == j):
                continue
            if (bases[i].commutes(bases[j])):
                comm_mat[i,j] = 1
    return comm_mat

In [28]:
num_wit_qubits = 7 # variable 'n' in paper
# For now, what the witness qubits are initialized to be doesn't matter (I THINK)
# as we are trying to minimize Val, given some optimal input states.
wit_qubits = np.zeros(num_wit_qubits, dtype=complex)

num_anc_qubits = 0 # variable 'm' in paper
anc_qubits = np.zeros(num_anc_qubits, dtype=complex)

num_t_gates = 4 # variable 't' in paper

num_meas_qubits = 1 # variable 'k' in paper
total_qubits = num_wit_qubits + num_anc_qubits

num_layers = 20

assert(num_layers > 0)

In [59]:
p_sum, _ = sim_circ(total_qubits, num_layers, num_t_gates)
print("Pauli Sum: " + str(p_sum))
print("Basis: " + str(p_sum.get_bases()))
#print("Succinct Basis: " + str(p_sum.succinct_basis(total_qubits)))
print("Min Eigenvalue + Eigenvector: " + str(p_sum.compute_min_eig_eigvec(total_qubits)))
#print(str(p_sum.compute_eig(total_qubits))) 

Pauli Sum: [(-0.3535533905932737+0j) ZXYYXIZ] + [(0.24999999999999992+0j) ZXIYIIX] + [(0.3535533905932737-0j) YIIZZZI] + [(-0.24999999999999992+0j) YIYZYZY] + [(-0.3535533905932737+0j) ZXXXZIZ] + [(0.24999999999999992-0j) ZXZXYIX] + [(0.3535533905932737+0j) YIZIXZI] + [(-0.24999999999999992+0j) YIXIIZY] + [(-0.24999999999999992+0j) IXXIYXY] + [(-0.24999999999999992+0j) XIZXIYX] + [(0.24999999999999992+0j) IXYZIXY] + [(-0.24999999999999992+0j) XIIYYYX]
Basis: [stim.PauliString("-ZXYYX_Z"), stim.PauliString("-i__Y_X_Y"), stim.PauliString("-iXXYXYZZ"), stim.PauliString("-ZXXXZ_Z"), stim.PauliString("+iZ_ZYZXX")]
Min Eigenvalue + Eigenvector: ((-1.4546564555882122+1.3347376556716105e-16j), array([-7.67395915e-05-5.52296462e-04j,  3.73135957e-02+3.83299319e-02j,
        3.13501035e-02-4.76097487e-04j, -2.63677968e-16-2.10837231e-01j,
        1.09009912e-01+5.33460308e-02j, -3.64105480e-02+4.14753297e-02j,
       -3.07544870e-02-5.00540680e-02j,  1.52371218e-02+3.50028632e-02j,
       -2.950

In [60]:
#print(p_sum.find_summand_bases_mapping())
succinct_p_sum, succinct_basis = p_sum.succinct_p_sum(total_qubits)
succinct_p_sum.set_bases(succinct_basis)
print("Succinct Pauli Sum: " + str(succinct_p_sum))
print("Succinct Basis: " + str(succinct_p_sum.get_bases()))
print("Min Eigenvalue + Eigenvector: " + str(succinct_p_sum.compute_min_eig_eigvec(len(p_sum.get_bases()))))
#print("Eigenvalues + Eigenvectors: " + str(succinct_p_sum.compute_eig(len(p_sum.get_bases()))))

{'+ZXYYX_Z': GF([[1],
    [0],
    [0],
    [0],
    [0]], order=2), '+ZX_Y__X': GF([[1],
    [1],
    [0],
    [0],
    [0]], order=2), '-Y__ZZZ_': GF([[1],
    [0],
    [1],
    [0],
    [0]], order=2), '+Y_YZYZY': GF([[1],
    [1],
    [1],
    [0],
    [0]], order=2), '-ZXXXZ_Z': GF([[0],
    [0],
    [0],
    [0],
    [0]], order=2), '-ZXZXY_X': GF([[0],
    [1],
    [0],
    [0],
    [0]], order=2), '+Y_Z_XZ_': GF([[0],
    [0],
    [1],
    [0],
    [0]], order=2), '-Y_X__ZY': GF([[0],
    [1],
    [1],
    [0],
    [0]], order=2), '+_XX_YXY': GF([[0],
    [0],
    [0],
    [0],
    [0]], order=2), '+X_ZX_YX': GF([[0],
    [0],
    [1],
    [0],
    [0]], order=2), '+_XYZ_XY': GF([[1],
    [0],
    [0],
    [0],
    [0]], order=2), '+X__YYYX': GF([[1],
    [0],
    [1],
    [0],
    [0]], order=2)}
(-1+0j)
-1j
1j
(-1+0j)
(1+0j)
(-0-1j)
(-0-1j)
(1+0j)
(1+0j)
(-0-1j)
(-1+0j)
1j
Succinct Pauli Sum: [(0.3535533905932737-0j) ZIIII] + [-0.24999999999999992j YZIII] + [0.353553390593273

### Steps in code
* Find bases that generate particular Pauli Summand:
    * Issue with simply doing $\alpha_1b_0b_1$ is that although $b_0b_1$ gives me the Pauli string that I want, they introduce some undesirable phase into it. This phase needs to be counteracted in the 'succinct_p_sum'
    * Need to check that in general, multiplying by the necessary phase to make $\pm i b_0b_1$ gives us what we want, for choice of either $+i$ or $-i$
    * Also need to check that multiplying the new bases yields the same phases
* Look at 'succinct_p_sum_summand' code

In [41]:
from qiskit.quantum_info.operators.symplectic import Pauli

b1 = Pauli("-YYIIYIZ")
b2 = Pauli("+XIZYZIY")
b3 = Pauli("+iIXXZXZI")
b4 = Pauli("-IZIIZXX")

mb1 = b1.to_matrix()
mb2 = b2.to_matrix()
mb3 = b3.to_matrix()
mb4 = b4.to_matrix()

sb1 = Pauli("ZIII")
sb2 = Pauli("XZII")
sb3 = Pauli("IXZI")
sb4 = Pauli("XXXZ")

msb1 = sb1.to_matrix()
msb2 = sb2.to_matrix()
msb3 = sb3.to_matrix()
msb4 = sb4.to_matrix()

b1b2 = b1.dot(b2)
b2b3 = b2.dot(b3)
b1b3 = b1.dot(b3)
b1b4 = b1.dot(b4)
b2b4 = b2.dot(b4)
b3b4 = b3.dot(b4)

sb1sb2 = sb1.dot(sb2)
sb2sb3 = sb2.dot(sb3)
sb1sb3 = sb1.dot(sb3)
sb1sb4 = sb1.dot(sb4)
sb2sb4 = sb2.dot(sb4)
sb3sb4 = sb3.dot(sb4)

print(b1b2, b2b3, b1b3, b1b4, b2b4, b3b4)
print(sb1sb2, sb2sb3, sb1sb3, sb1sb4, sb2sb4, sb3sb4)



iZYZYXIX XXYXYZY iYZXZZZZ -iYXIIXXY iXZZYIXZ -IYXZYYX
iYZII iXYZI ZXZI iYXXZ iIYXZ iXIYZ


In [42]:
p1 = Pauli("ZIII")
p2 = Pauli("XZII")
p3 = Pauli("iXYZI")
p4 = Pauli("XXXZ")
p5 = Pauli("iXIYZ")
p6 = Pauli("+iIYXZ")

t = (-1) * (0.7071067811865475-0j) * p1.to_matrix() + (0.3535533905932737+0j) * p2.to_matrix() + (-1j) * (0.3535533905932737-0j) * p3.to_matrix() + (-1) * (-0.3535533905932737+0j) * p4.to_matrix()+ (1j) * (0.3535533905932737+0j) * p5.to_matrix()
print(min(np.linalg.eig(t)[0]))


(-1-9.813610577122677e-17j)


In [16]:
old_basis = p_sum.get_bases()
print(old_basis, succinct_basis)
comm_mat_1 = get_commutation_matrix(old_basis)
comm_mat_2 = get_commutation_matrix(succinct_basis)
print(comm_mat_1)
print(comm_mat_2)
print(comm_mat_1 == comm_mat_2)

[stim.PauliString("-iZZXYZYY"), stim.PauliString("+ZX_XXY_"), stim.PauliString("+ZY_XXY_")] [stim.PauliString("+Z__"), stim.PauliString("+XZ_"), stim.PauliString("+XXZ")]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


In [586]:
num_iter = 100
file = open('diff_e_val.txt', 'w')
write_str = ""
for _ in range(num_iter):
    p_sum, _ = sim_circ(total_qubits, num_layers, num_t_gates)
    file.write("Pauli Sum: " + str(p_sum) + '\n')
    file.write("Basis: " + str(p_sum.get_bases()) + '\n')
    file.write("Succinct Basis: " + str(p_sum.succinct_basis(total_qubits)) + '\n')
    min_eig = p_sum.compute_min_eig_eigvec(total_qubits)
    succinct_p_sum, _ = p_sum.succinct_p_sum(total_qubits)
    file.write("Succinct Pauli Sum: " + str(succinct_p_sum) + '\n')
    succinct_min_eig = succinct_p_sum.compute_min_eig_eigvec(len(p_sum.get_bases()))
    file.write("Min Eigenvalue + Eigenvector: " + str(min_eig) + '\n')
    file.write("Succinct Min Eigenvalue + Eigenvector: " + str(succinct_min_eig) + '\n')
    file.write('\n')

file.close()

{'-_Z_Y_ZZYYY': GF([[1],
    [0],
    [0]], order=2), '-YZ_XXZYYZY': GF([[0],
    [1],
    [0]], order=2), '+YXYXYXZXYZ': GF([[0],
    [0],
    [1]], order=2)}
{'-_YZXYZ_Z__': GF([[1],
    [0],
    [0]], order=2), '-XXZZYYZ_XY': GF([[0],
    [1],
    [0]], order=2), '-_YYY_X_ZXX': GF([[0],
    [0],
    [1]], order=2)}
{'+ZZ_ZZXYXYX': GF([[1]], order=2)}
{'+ZY_ZYX_X_Y': GF([[1]], order=2)}
{'-__YY_ZXY_X': GF([[1],
    [0],
    [0]], order=2), '+YYYXYXZYZ_': GF([[0],
    [1],
    [0]], order=2), '+_YX_ZZYZZ_': GF([[0],
    [0],
    [1]], order=2)}


KeyboardInterrupt: 